In [1]:
import requests
import urllib.request
from urllib.parse import quote
import numpy as np
from tqdm import tqdm
#import contextlibr
import collections
import os
import json
import tqdm
from bs4 import BeautifulSoup as bs
import time
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import collections
import pandas as pd

### Set up spotify scrapping functions

In [2]:
token = 'BQAU7cwi4gE4azyohgwWGhdi9Q6azGYvEwdnM0Iy73qCaHl27LLiwNzSX11f0wOZUrzOxqI1duCcHVbnvKdYLsBVth-IYCATaqtWWrQex2o4bK7zwiEXHWGqWYUh6qRPIox0vKp2uiO8j_P9lWuwGxq25Bw'

header = {
    'Authorization': 'Bearer ' + token,
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

In [3]:
final_json = []
intermediary_results = []

In [4]:
def song_features(song_id):
    query = 'audio-features/' + str(song_id)
    r = requests.request('get', 'https://api.spotify.com/v1/' + query, headers=header)
    return r.json()


def get_rateYourMusic_tuples(filenames, table_index=3) :
    
    #song year, artist and title
    searches = []
    
    for f in filenames :

        res = bs(open(f),"html5lib")
        for t in res.findAll(attrs={'class': "mbgen"})[table_index].findAll("tr")[1:] :
                        
            if len(t.findAll("td")) >= 3 :
                
                
            
                year = t.findAll("td")[0].text
                artist = t.findAll("td")[1].text
                title = t.findAll("td")[2].text

                if '/' in title :
                    searches.append([year, artist, title.split('/')[0].strip()])
                    searches.append([year, artist, title.split('/')[1].strip()])

                else :
                    searches.append([year, artist, title])

    return searches

def search_tuples(genre, searches) :
    
    found = 0
    all_info = []

    for e, s in tqdm.tqdm(enumerate(searches)) :
        result  = requests.request('get', 'https://api.spotify.com/v1/search?q=album:'+ quote(s[2]) + "%20artist:" + quote(s[1]) + "&type=track", headers=header).json()
        time.sleep(0.3)
        items = result['tracks']['items']
        
        
        #We found the song
        if len(items) != 0 and ('error' not in result.keys()) :           
            
            album_type = result['tracks']['items'][0]['album']['album_type']
            
            #the song does not come from a compilation
            if album_type != 'compilation' :
            
                album = result['tracks']['items'][0]['album']['name']
                name = result['tracks']['items'][0]['name']

                song_id = result['tracks']['items'][0]['id']   
                song_release = items[0]['album']['release_date']
                features = song_features(song_id)

                if 'error' not in features.keys() :

                    found += 1

                    #get the preview url
                    result_preview  = requests.request('get', "https://api.spotify.com/v1/tracks/" + str(song_id), headers=header).json()
                    time.sleep(0.3)
                    preview = result_preview['preview_url']

                    if preview is None :
                        preview = ""

                    
                    cover = ""

                    if len(items[0]['album']['images']) != 0 :                
                        cover = items[0]['album']['images'][0]['url']            
                    else :
                        cover = "https://vignette.wikia.nocookie.net/uncyclopedia/images/a/a0/Questionmark.jpg/revision/latest?cb=20090616065802&format=original"

                    artists = s[1]



                    song_dict = {'year' : song_release,
                                 'artists' : artists,
                                 'name' : name,
                                 'cover' : cover,
                                 'album' : album,
                                 'genre' : genre,
                                'danceability' : features['danceability'],
                                'energy' : features['energy'],
                                'key' : features['key'],
                                'loudness' : features['loudness'],
                                'mode' : features['mode'],
                                'speechiness' : features['speechiness'],
                                'acousticness' : features['acousticness'],
                                'instrumentalness' : features['instrumentalness'],
                                'liveness' : features['liveness'],
                                'valence' : features['valence'],
                                'tempo' : features['tempo'],
                                'duration_ms' : features['duration_ms'],
                                'preview' : preview}


                    all_info.append(song_dict)
                    
            
                
    print("found : ", found , ", (" + str(100*found/len(searches)) + "%) for genre " + genre)
    return all_info


# RateYourMusic

### Download Contemporary RnB titles (93 songs)

From Itunes (70)

In [5]:
# get names from itunes
url = 'https://itunes.apple.com/search?term=song&genreId=1136&limit=200'

response = requests.get(url)
data = response.json()

found = 0

for e in tqdm.tqdm(data['results']):
    result  = requests.request('get', 'https://api.spotify.com/v1/search?q='+ quote(e['trackName']) + "%20artist:" + quote(e['artistName']) + "&type=track", headers=header).json()
    
    items = result['tracks']['items']
    
    
    #We found the song
    if len(items) != 0 and ('error' not in result.keys()):
        
        found += 1
    
        song_id = result['tracks']['items'][0]['id']    
        features = song_features(song_id)
        
        #get the preview url
        result_preview  = requests.request('get', "https://api.spotify.com/v1/tracks/" + str(song_id), headers=header).json()
        time.sleep(0.3)
        preview = result_preview['preview_url']

        if preview is None :
            preview = ""

        year = e['releaseDate']
        cover = e['artworkUrl100']
        artists = " and ".join([a['name'] for a in result['tracks']['items'][0]['artists']])
        name = e['trackName']
        album = result['tracks']['items'][0]['album']['name']



        song_dict = {'year' : year,
                     'artists' : artists,
                     'name' : name,
                     'cover' : cover,
                     'album' : album,
                     'genre' : "contemporary-rnb",
                    'danceability' : features['danceability'],
                    'energy' : features['energy'],
                    'key' : features['key'],
                    'loudness' : features['loudness'],
                    'mode' : features['mode'],
                    'speechiness' : features['speechiness'],
                    'acousticness' : features['acousticness'],
                    'instrumentalness' : features['instrumentalness'],
                    'liveness' : features['liveness'],
                    'valence' : features['liveness'],
                    'tempo' : features['tempo'],
                    'duration_ms' : features['duration_ms'],
                    'preview' : preview}


        final_json.append(song_dict)
        
intermediary_results = final_json.copy()

  0%|          | 0/94 [00:00<?, ?it/s]


KeyError: 'tracks'

### Download british RnB (148 songs)

In [ ]:
file_names_brit = []
for nb in range(250) :
    file_name = "data/british_rnb/british_rnb_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_brit.append(file_name)

#break the search in two
brit_tuples = get_rateYourMusic_tuples(file_names_brit)    


final_json.extend(search_tuples("british-rnb", brit_tuples[:int(len(brit_tuples)/2)]))
intermediary_results = final_json.copy()

final_json.extend(search_tuples("british-rnb", brit_tuples[int(len(brit_tuples)/2):]))
intermediary_results = final_json.copy()

### Download alternative RnB (640 songs)

In [ ]:
file_names_alt = []
for nb in range(250) :
    file_name = "data/alternative_rnb/alt_songs_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_alt.append(file_name)

alt_tuples = get_rateYourMusic_tuples(file_names_alt)

        
final_json.extend(search_tuples("alternative-rnb", alt_tuples))
intermediary_results = final_json.copy()

### Download new jack swing RnB (146 songs)

In [ ]:
file_names_nj = []
for nb in range(250) :
    file_name = "data/new_jack_swing_rnb/njs_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_nj.append(file_name)

njs_tuples = get_rateYourMusic_tuples(file_names_nj)
        
final_json.extend(search_tuples("new-jack-swing-rnb", njs_tuples))
intermediary_results = final_json.copy()

### Download new Orleans RnB (247 songs)

In [ ]:
file_names_no = []
for nb in range(250) :
    file_name = "data/new_orleans_rnb/no_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_no.append(file_name)
        
no_tuples = get_rateYourMusic_tuples(file_names_no)

final_json.extend(search_tuples("new-orleans-rnb", no_tuples))
intermediary_results = final_json.copy()

### Download Jump Blues (159 songs)

In [ ]:
file_names_jb = []
for nb in range(250) :
    file_name = "data/jump_blues_rnb/jp_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_jb.append(file_name)

jb_tuples = get_rateYourMusic_tuples(file_names_jb)
        
final_json.extend(search_tuples("jump-blues", jb_tuples))
intermediary_results = final_json.copy()

### Download blue-eyed soul (622 songs)

In [ ]:
file_names_bes = []
for nb in range(250) :
    file_name = "data/blue_eyed_soul/blue_eyed_soul_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_bes.append(file_name)

bes_tuples = get_rateYourMusic_tuples(file_names_bes)
        
final_json.extend(search_tuples("blue-eyed-soul", bes_tuples))

In [ ]:
#Dump everything into the data
json.dump(final_json, open("rnb_data.json", 'w'))

# AllMusic

### Contemporary rnb (720)

In [ ]:
cont_tuples = json.load(open("data/All_Music/parsed/am_contemporary_rnb.json"))
final_json.extend(search_tuples("contemporary-rnb", cont_tuples))
intermediary_results = final_json.copy()

### early rnb (198)

In [ ]:
early_tuples = json.load(open("data/All_Music/parsed/am_early_rnb.json"))
final_json.extend(search_tuples("early-rnb", early_tuples))
intermediary_results = final_json.copy()

### motown (86)

In [ ]:
mo_tuples = json.load(open("data/All_Music/parsed/am_motown.json"))
final_json.extend(search_tuples("motown-rnb", mo_tuples))
intermediary_results = final_json.copy()

In [ ]:
json.dump(final_json, open("rnb_data.json", 'w'))

# visualize number of songs per year

In [ ]:
import collections

#all possible genres
genres = ['contemporary-rnb', 'british-rnb', 'new-orleans-rnb', 
          'alternative-rnb', 'new-jack-swing-rnb', 'blue-eyed-soul', 'jump-blues']

#compute earliest and latest year globally
all_years = [d['year'] for d in final_json if (d['year'] != '')]
min_all_years, max_all_years = int(min(all_years)), int(max(all_years)) + 1

#plot settings
width = 1
figure(figsize=(15,10))
plots = []

#initialize all years to zero
seen_years = collections.Counter()
for y in range(min_all_years, max_all_years) :
        seen_years[str(y)] = 0



for g in genres :

    #years of genre songs
    genre_years = [d['year'] for d in final_json if ((d["genre"] == g) and (d['year'] != ''))]
    
    #create counter
    years_per_genre = collections.Counter()
    
    
    #put 0 as default value for all years
    for y in range(min_all_years, max_all_years) :
        years_per_genre[str(y)] = 0
        
    #count values
    years_per_genre.update(genre_years)
    
    #create plot variables
    labels, values = zip(*years_per_genre.items())
    labels_all, values_all = zip(*seen_years.items())
    indexes = np.arange(len(labels))    
    
    #plot with bottom
    p = plt.bar(indexes, values, width, bottom=values_all)    
    plots.append(p)
    
    #update seen years counter
    seen_years.update(genre_years)

#settings for axes
plt.xticks(indexes + width * 0.5, labels, rotation=90)
plt.xlabel("release year")
plt.ylabel("number of songs")
plt.ylim(ymax = 160, ymin = 0)

#general plot settings
plt.title("number of RnB songs per genre and per year")
plt.legend(plots, genres)

# plot
plt.savefig("nb_of_songs.png")
plt.show()


# compute discriminative coefficients for each feature and each year 

In [ ]:
[f for f in final_json_bis if f['genre'] == 'contemporary-rnb']

In [ ]:
years = range(1900, 2050)
best_features = dict()

for y in years :
    
    # load data
    names = ['energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    targets = np.array([c['genre'] for c in final_json if c['year'] == str(y)])
    
    if len(set(targets)) > 1:

        features = np.array([c['danceability'] for c in final_json if c['year'] == str(y)])
        
        
        for n in names :
            features = np.vstack((features, np.array([c[n] for c in final_json if c['year'] == str(y)])))


        model = LogisticRegression()
        rfe = RFE(model, 2)
        fit = rfe.fit(features.T, targets)

        best_for_year = sorted([c[0] for c in zip(['danceability'] + names, fit.support_) if c[1]])
        best_features[y] = best_for_year
        
best_features